In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!ls /content/drive/MyDrive/Training_Essay_Data00.csv

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight

In [4]:
import tensorflow as tf

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np
from transformers import BertTokenizer

# Load and preprocess the data
def load_data():
    """Load data from a CSV file with 'text' and 'label' columns."""
    df = pd.read_csv('Training_Essay_Data.csv', encoding='latin1')
    texts = df['text'].tolist()
    labels = df['generated'].tolist()
    return texts, labels

def preprocess_data(texts, tokenizer, max_length=128):
    """Tokenize texts and prepare labels for BERT input."""
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    return input_ids, attention_mask

# Define the BERT model
def create_model():
    """Create and compile a BERT-based text classification model."""
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')


    input_ids = tf.keras.layers.Input(shape=(128,), name='input_ids', dtype='int32')
    attention_mask = tf.keras.layers.Input(shape=(128,), name='attention_mask', dtype='int32')

    bert_output = bert_model([input_ids, attention_mask])
    cls_token = bert_output.last_hidden_state[:, 0, :]
    output = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Load BERT tokenizer
#tokenizer = TFBertTokenizer.from_pretrained('bert-base-uncased')

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess data
#file_path = 'data.csv'  # Path to your data file
texts, labels = load_data()
input_ids, attention_mask = preprocess_data(texts, tokenizer)

# Convert labels to Tensor
labels = tf.convert_to_tensor(labels)



# Perform k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold = 1
for train_index, val_index in kf.split(input_ids):
    print(f"\nFold {fold}/{k}")

    # Split data into train and validation sets
    train_input_ids, val_input_ids = tf.gather(input_ids, train_index), tf.gather(input_ids, val_index)
    train_attention_mask, val_attention_mask = tf.gather(attention_mask, train_index), tf.gather(attention_mask, val_index)
    train_labels, val_labels = tf.gather(labels, train_index), tf.gather(labels, val_index)

    # Create and compile the model
    model = create_model()
    model.summary()

    # Define callbacks for monitoring and saving the model
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'best_model_fold_{fold}.keras', save_best_only=True, monitor='val_loss')

    # Train the model
    history = model.fit(
        [train_input_ids, train_attention_mask],
        train_labels,
        validation_data=([val_input_ids, val_attention_mask], val_labels),
        epochs=5,
        batch_size=32,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Evaluate the model on the validation set
    val_predictions = model.predict([val_input_ids, val_attention_mask])
    val_predictions = (val_predictions > 0.5).astype(int)

    # Print classification report and accuracy
    print("Classification Report:\n", classification_report(val_labels, val_predictions))
    print("Accuracy Score:", accuracy_score(val_labels, val_predictions))

    fold += 1

# Save the final model
model.save('final_model.keras')



Fold 1/5



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

C:\Users\Ahmedma\AppData\Local\anaconda3\Lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


689/689 [==============================] - 8505s 12s/step - loss: 0.0511 - accuracy: 0.9816 - val_loss: 0.0675 - val_accuracy: 0.9766
Epoch 2/5
689/689 [==============================] - 8329s 12s/step - loss: 0.0109 - accuracy: 0.9970 - val_loss: 0.0221 - val_accuracy: 0.9926
Epoch 3/5
689/689 [==============================] - 8299s 12s/step - loss: 0.0063 - accuracy: 0.9978 - val_loss: 0.0247 - val_accuracy: 0.9940
Epoch 4/5
689/689 [==============================] - 8315s 12s/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.0118 - val_accuracy: 0.9964
Epoch 5/5
173/173 [==============================] - 604s 3s/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3502
           1       0.99      1.00      1.00      2006

    accuracy                           1.00      5508
   macro avg       1.00      1.00      1.00      5508
weighted avg       1.00      1.00      1.00      5508

Accuracy Score: 0

...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_4 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                     

C:\Users\Ahmedma\AppData\Local\anaconda3\Lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


689/689 [==============================] - 8683s 13s/step - loss: 0.0530 - accuracy: 0.9809 - val_loss: 0.0136 - val_accuracy: 0.9949
Epoch 2/5
689/689 [==============================] - 8476s 12s/step - loss: 0.0123 - accuracy: 0.9966 - val_loss: 0.0180 - val_accuracy: 0.9947
Epoch 3/5
689/689 [==============================] - 8415s 12s/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.0165 - val_accuracy: 0.9956
Epoch 4/5
173/173 [==============================] - 615s 4s/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3468
           1       0.99      0.99      0.99      2039

    accuracy                           0.99      5507
   macro avg       0.99      0.99      0.99      5507
weighted avg       0.99      0.99      0.99      5507

Accuracy Score: 0.9949155620119847


C:\Users\Ahmedma\AppData\Local\anaconda3\Lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define model architecture
def create_model():
    input_ids = tf.keras.layers.Input(shape=(128,), name='input_ids', dtype='int32')
    attention_mask = tf.keras.layers.Input(shape=(128,), name='attention_mask', dtype='int32')
    bert_output = bert_model([input_ids, attention_mask])
    cls_token = bert_output.last_hidden_state[:, 0, :]
    output = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Data preparation
def prepare_data(data, tokenizer):
    encodings = tokenizer(list(data['text'].values), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': tf.convert_to_tensor(encodings['input_ids']),
        'attention_mask': tf.convert_to_tensor(encodings['attention_mask'])
    }

# Load your data
data = pd.read_csv('path_to_data.csv')  # Adjust the path to your dataset
X = prepare_data(data, tokenizer)
y = data['generated'].values

# Cross-validation
skf = StratifiedKFold(n_splits=5)
accuracies, precisions, recalls, aucs = [], [], [], []

for train_index, val_index in skf.split(X['input_ids'], y):
    X_train = {key: val[train_index] for key, val in X.items()}
    X_val = {key: val[val_index] for key, val in X.items()}
    y_train, y_val = y[train_index], y[val_index]

    model = create_model()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=16, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

    y_pred = (model.predict(X_val) > 0.5).astype("int32")
    accuracies.append(accuracy_score(y_val, y_pred))
    precisions.append(precision_score(y_val, y_pred))
    recalls.append(recall_score(y_val, y_pred))
    aucs.append(roc_auc_score(y_val, y_pred))

# Evaluation
print(f'Accuracy: {np.mean(accuracies):.4f}')
print(f'Precision: {np.mean(precisions):.4f}')
print(f'Recall: {np.mean(recalls):.4f}')
print(f'AUC: {np.mean(aucs):.4f}')
